In [ ]:
!pip install googlemaps

In [ ]:
from google.colab import userdata
import googlemaps
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Finding the hospital count by simply querying google maps

In [ ]:
county_data = pd.read_csv('/content/us-county-boundaries.csv', delimiter=';')

In [ ]:
def get_hospitals_in_county(county_name, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=hospitals+in+{county_name}+county&key={api_key}"
    response = requests.get(url)
    results = response.json().get('results', [])
    return len(results)

In [ ]:
api_key = userdata.get('googlemaps')
county_data['hospital_count'] = county_data['NAME'].apply(lambda x: get_hospitals_in_county(x, api_key))

In [ ]:
county_data[['NAMELSAD', 'hospital_count']].to_csv('hospitals_per_county.csv', index=False)

### Finding transit options around each hospital

In [ ]:
county_data = pd.read_csv('/content/us-county-boundaries.csv', delimiter=';')

In [ ]:
def get_hospitals_in_county(county_name, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=hospitals+in+{county_name}+county&key={api_key}"
    response = requests.get(url)
    results = response.json().get('results', [])
    return results

In [ ]:
def get_transit_options(hospital_location, api_key):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={hospital_location}&destination={hospital_location}&mode=transit&key={api_key}"
    response = requests.get(url)
    routes = response.json().get('routes', [])
    return routes


In [ ]:
api_key = userdata.get('googlemaps')
gmaps = googlemaps.Client(key=api_key)
transit_data = []

for county in county_data['NAME']:
    hospitals = get_hospitals_in_county(county, api_key)
    for hospital in hospitals:
        hospital_location = f"{hospital['geometry']['location']['lat']},{hospital['geometry']['location']['lng']}"
        transit_options = gmaps.places_nearby(location=hospital_location, radius=1000, type='transit_station')
        transit_data.append({
            'county': county,
            'hospital_name': hospital['name'],
            'transit_options': [transit_options['results'][x]['name'] for x in range(len(transit_options['results']))]
        })

transit_df = pd.DataFrame(transit_data)
transit_df['num_transit_options'] = transit_df['transit_options'].apply(len)

In [ ]:
transit_by_county = transit_df.groupby('county')['num_transit_options'].mean().reset_index()

plt.figure(figsize=(12, 6))

sns.barplot(x='county', y='num_transit_options', data=transit_by_county)

mean_transit_options = transit_by_county['num_transit_options'].mean()
first_quartile = transit_by_county['num_transit_options'].quantile(0.25)
third_quartile = transit_by_county['num_transit_options'].quantile(0.75)

plt.axhline(y=mean_transit_options, color='red', linestyle='--', label='Mean')
plt.axhline(y=first_quartile, color='green', linestyle='--', label='1st Quartile')
plt.axhline(y=third_quartile, color='blue', linestyle='--', label='3rd Quartile')

plt.legend()
plt.style.use('fivethirtyeight')
plt.xlabel('County')
plt.ylabel('Average Number of Transit Options Near Hospitals')
plt.title('Average Transit Options Near Hospitals per County')
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
transit_df.to_csv('transit_options_around_hospitals.csv', index=False)

In [ ]:
imperial_transit_df = transit_df[transit_df['county'] == 'Imperial']
imperial_transit_by_hospital = imperial_transit_df.groupby('hospital_name')['num_transit_options'].mean().reset_index()

plt.figure(figsize=(12, 6))

sns.barplot(x='hospital_name', y='num_transit_options', data=imperial_transit_by_hospital)

mean_transit_options = imperial_transit_by_hospital['num_transit_options'].mean()
plt.axhline(y=mean_transit_options, color='red', linestyle='--', label='Mean')

plt.legend()
plt.xlabel('Hospital')
plt.ylabel('Transit Options Near Hospitals in Imperial County')
plt.title('Transit Options Near Hospitals')
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
imperial_transit_df = transit_df[transit_df['county'] == 'San Diego']
imperial_transit_by_hospital = imperial_transit_df.groupby('hospital_name')['num_transit_options'].mean().reset_index()

plt.figure(figsize=(12, 15))

sns.barplot(x='hospital_name', y='num_transit_options', data=imperial_transit_by_hospital)

mean_transit_options = imperial_transit_by_hospital['num_transit_options'].mean()
plt.axhline(y=mean_transit_options, color='red', linestyle='--', label='Mean')

plt.legend()
plt.xlabel('Hospital')
plt.ylabel('Transit Options Near Hospitals in San Diego County')
plt.title('Transit Options Near Hospitals')
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()